In [1]:
import pandas as pd
import numpy as np
from sklearn import cross_validation, linear_model
from sklearn.ensemble import RandomForestClassifier
from sklearn.grid_search import GridSearchCV

In [5]:
from data_pipeline.data_clean import read_file

In [5]:
filename = 'data_collection/match_history_sample.json'

In [7]:
# match_json_array = []
with open(filename) as f:
#     print sum(1 for _ in f)
   arr = [next(f) for x in xrange(300)]
#     for linze in f:
#         match_json_array.append(line)

In [8]:
t = map(lambda x: pd.read_json(x), arr)


In [9]:
df = pd.concat(t,ignore_index=True)

In [10]:
df = pd.DataFrame(df.matches.tolist())

In [11]:
df_10player = df[df['human_players']==10]

In [12]:
def match_features(players):
    vec = np.zeros(224, dtype=np.int)
    loc = np.append(np.zeros(5, dtype=np.int), np.ones(5, dtype=np.int)*112)
    loc+=map(lambda x: x['hero_id']-1, players)
    vec[loc]=1
    return vec

def matches_features(df):
    df_feature = df['players'].apply(lambda x: match_features(x))
    X = pd.DataFrame(df_feature.tolist())
    y = df['radiant_win']
    return X, y

In [13]:
X, y = matches_features(df_10player)

# Random Forest

In [110]:
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size=0.8)

In [21]:
# Set the parameters by cross validation

tuned_parameters = [
                    {'n_estimators':[100, 1000, 10000]},
                    {'max_features':[5, 'auto', None]},
                    {'min_samples_leaf':[1, 50, 150]}
                    ]

In [16]:
clf_rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=50)

In [17]:
clf = GridSearchCV(clf_rf, tuned_parameters, cv=5)

In [18]:
clf.fit(X_train, y_train)

/home/ubuntu/anaconda2/lib/python2.7/site-packages/sklearn/ensemble/forest.py:403: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "


GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
            oob_score=True, random_state=50, verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid=[{'n_estimators': [100, 1000, 10000]}, {'max_features': [5, 'auto', None]}, {'min_samples_leaf': [1, 50, 150]}],
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [19]:
clf.best_params_                                            

{'n_estimators': 10000}

#### Random Forest

In [16]:
clf_rf =RandomForestClassifier(n_estimators=10000,n_jobs=-1, min_samples_leaf=50, random_state=50)

In [17]:
cross_validation.cross_val_score(clf_rf, X, y, cv=5)

KeyboardInterrupt: 

#### Logistics

In [15]:
clf_log_r = linear_model.LogisticRegression()
cross_validation.cross_val_score(clf_log_r, X, y, cv=5)

array([ 0.5940206 ,  0.60404703,  0.59921612,  0.59489517,  0.59963537])

In [117]:
# param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000] }
# clf = GridSearchCV(linear_model.LogisticRegression(penalty='l2'), param_grid)
clf=GridSearchCV(cv=None,
                 estimator=linear_model.LogisticRegression(C=1.0, 
                                              intercept_scaling=1, 
                                              dual=False, 
                                              fit_intercept=True, 
                                              penalty='l2', 
                                              tol=0.0001),
                 param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]})

In [118]:
clf.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0),
       fit_params={}, iid=True, loss_func=None, n_jobs=1,
       param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]},
       pre_dispatch='2*n_jobs', refit=True, score_func=None, scoring=None,
       verbose=0)

In [119]:
clf.best_params_

{'C': 1}

In [ ]:
#### 

In [2]:
pwd

u'/home/hao/Repos/DSI/DOTA2_Recommendation-System/notebook'

In [3]:
cd ..

/home/hao/Repos/DSI/DOTA2_Recommendation-System


In [6]:
ls

data_collection/  data_pipeline/  file/  model/  notebook/  README.md  Test/


In [8]:
from model.dota_model import Dota2_model

In [9]:
model = Dota2_model()

In [10]:
X, y = model.csv_transform()

In [11]:
X = np.array(X)

In [12]:
X.shape

(54853, 220)

In [14]:
y[:5]

[True, True, True, True, False]

In [4]:
df = pd.read_csv('file/heroes_csv', index_col=[0])

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 219416 entries, 1 to 259999
Data columns (total 12 columns):
0              219416 non-null int64
1              219416 non-null int64
2              219416 non-null int64
3              219416 non-null int64
4              219416 non-null int64
5              219416 non-null int64
6              219416 non-null int64
7              219416 non-null int64
8              219416 non-null int64
9              219416 non-null int64
radiant_win    219416 non-null bool
match_id       219416 non-null int64
dtypes: bool(1), int64(11)
memory usage: 20.3 MB


In [6]:
df.head()

,0,1,2,3,4,5,6,7,8,9,radiant_win,match_id
1,19,74,18,84,75,93,63,21,8,7,True,1936712411
2,57,99,11,8,76,21,44,28,5,59,True,1936756135
3,14,25,59,109,22,8,64,28,86,70,False,1936757580
4,74,106,112,53,4,39,70,92,8,67,False,1936768967
6,94,36,112,88,67,25,48,59,21,104,True,1936747708


In [76]:
with open ('file/heroes_csv') as f:
    a = [next(f) for i in range(5)]

In [77]:
a

[',0,1,2,3,4,5,6,7,8,9,radiant_win,match_id\n',
 '1,19,74,18,84,75,93,63,21,8,7,True,1936712411\n',
 '2,57,99,11,8,76,21,44,28,5,59,True,1936756135\n',
 '3,14,25,59,109,22,8,64,28,86,70,False,1936757580\n',
 '4,74,106,112,53,4,39,70,92,8,67,False,1936768967\n']

In [80]:
a[3].split(",")

['3',
 '14',
 '25',
 '59',
 '109',
 '22',
 '8',
 '64',
 '28',
 '86',
 '70',
 'False',
 '1936757580\n']

In [81]:
b = a[3].split(",")[1:-1]

In [82]:
b

['14', '25', '59', '109', '22', '8', '64', '28', '86', '70', 'False']

In [97]:
import ast

In [99]:
type(ast.literal_eval(b[-1]))

bool

In [17]:
type(b[-1])

str

In [18]:
bool(b[-1])

True

In [17]:
b

['19', '74', '18', '84', '75', '93', '63', '21', '8', '7', 'True']

In [18]:
c = [0, 0, 0, 0, 0, 113, 113, 113, 113, 113]

In [22]:
map(int, b[:-1]) + np.array(c)

array([ 19,  74,  18,  84,  75, 206, 176, 134, 121, 120])

In [25]:
np.array([0]*5 + [113] *5)

array([  0,   0,   0,   0,   0, 113, 113, 113, 113, 113])